<a href="https://colab.research.google.com/github/AbrahamKong/CMPE257-NLP_AlternusVera/blob/master/AlternusVera_Corpis_Style_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternus Vera 

-----

GitHub URL: 


### Liar Liar Pants on Fire Dataset Description 
- It has 3 files test, training and valid.
- Each file has 14 columns
    
    Column 1: the ID of the statement ([ID].json).
    
    Column 2: the label.
    
    Column 3: the statement.
    
    Column 4: the subject(s).
    
    Column 5: the speaker.
    
    Column 6: the speaker's job title.
    
    Column 7: the state info.
    
    Column 8: the party affiliation.
    
    Column 9-13: the total credit history count, including the current statement.
    
    Column 14: the context (venue / location of the speech or statement).

### Process 
- Load the Data
- Distillation Process
    - Data Cleaning and Text Preprocessing
    - Visualization
- **Feature 1 :** Corpus Style 
    

### Team Contributions example:

|Features  |  Member |
|-----|-----|
| Feature name(s)                         |  Member name(s) |  
| Feature name(s)                 |  Member name(s) | 
| Feature name(s)                   |  Member name(s)  |   
| Feature name(s)                             |  Member name(s) |



#### What did I try and What worked? 

> Explain your work ...

#### What did not work?

> Explain your work ...


#### What alternatives did you try?

> Explain your work 

----

# Data Pre-process

### Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
file_path_data = "/content/drive/MyDrive/SJSU/CMPE 257: Machine Learning/NLP/AlternusVera/input_data"

In [4]:
your_team="Data Miners/NLP-Alternus Vera/input_data"
# file_path_data ="/content/drive/MyDrive/ML-Spring-2022/"+your_team
import os
os.chdir(file_path_data)

### Reading the Data

In [5]:
# !pip install selenium
# !pip install newspaper3k
# ! pip install beautifulsoup4
# ! pip install requests

In [6]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from scipy import sparse
# Code source: https://degravek.github.io/project-pages/project1/2017/04/28/New-Notebook/
# Dataset from Chakraborty et al. (https://github.com/bhargaviparanjape/clickbait/tree/master/dataset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
# Read the test, training and valid data from files
# Header = 0 indicates that the first line of the file contains column names,
# As there is no Header, create a column names for each column in the dataset
# delimiter = \t indicates that the fields are seperated by tabs, and 


# test_filename = 'input_data/dataset/test.tsv'
# train_filename = 'input_data/dataset/train.tsv'
# valid_filename = 'input_data/dataset/valid.tsv'

test_filename = file_path_data + '/test.tsv'
train_filename = file_path_data + '/train.tsv'
valid_filename = file_path_data + '/valid.tsv'

colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

train_news = pd.read_csv(train_filename, sep='\t', names = colnames, error_bad_lines=False)
test_news = pd.read_csv(test_filename, sep='\t', names = colnames, error_bad_lines=False)
valid_news = pd.read_csv(valid_filename, sep='\t', names = colnames, error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


#### tweets from Donald Trump

In [8]:
# Display check the dimensions and the first 2 rows of the file.

print('train dim:',train_news.shape, 'test dim:', test_news.shape)
train_news.iloc[0:2]

train dim: (10240, 14) test dim: (1267, 14)


,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.


In [9]:
tweets_filename = file_path_data + '/trump_tweets.csv'

trump_tweets = pd.read_csv(tweets_filename)

In [10]:
trump_tweets = trump_tweets.drop(columns=['id', 'isRetweet', 'isDeleted', 'device', 'favorites', 'retweets', 'isFlagged'])

In [11]:
trump_tweets = trump_tweets[trump_tweets["text"].str.startswith("http")==False]
# trump_tweets = trump_tweets[trump_tweets["text"].str.startswith("RT")==False]

In [12]:
trump_tweets

,text,date
0,Republicans and Democrats have both created ou...,2011-08-02 18:07:48
1,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50
2,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47
3,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58
4,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59
...,...,...
56566,RT @RandPaul: I don’t know why @JoeBiden think...,2020-10-23 03:46:25
56567,RT @EliseStefanik: President @realDonaldTrump ...,2020-10-23 03:42:05
56568,RT @TeamTrump: LIVE: Presidential Debate #Deba...,2020-10-23 01:03:58
56569,Just signed an order to support the workers of...,2020-10-22 21:04:21


## Data Cleaning and Text Preprocessing 

*Steps included in the preprocessing:*
- Remove Special Characters and Punctuations
- Lower case the news
- Tokenization
- Remove Stop Words
- Lemmatization
- Stemming 
- Spell Check 

###  Putting It All Together 

To make the code reusable, we need to create a function that can be called many times.

In [13]:
def cleaning(raw_news):
    # import nltk
    # nltk.download('punkt')
    # nltk.download('wordnet')
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)

In [14]:
import time
# clean training and test data 
# create new column "tokenized"
t1 = time.time()

# Add the processed data to the original data. 
# Perhaps using apply function would be more elegant and concise than using for loop
import nltk
nltk.download('punkt')
nltk.download('wordnet')
train_news['clean'] = train_news["headline_text"].apply(cleaning) 

t2 = time.time()
print("\nTime to clean, tokenize and stem train data: \n", len(train_news), "news:", (t2-t1)/60, "min")

t1 = time.time()
test_news['clean'] = test_news["headline_text"].apply(cleaning)

t2 = time.time()
print("\n\nTime to clean, tokenize and stem test data: \n", len(test_news), "news:", (t2-t1)/60, "min")

t1 = time.time()
valid_news['clean'] = valid_news["headline_text"].apply(cleaning)

t2 = time.time()
print("\n\nTime to clean, tokenize and stem valid data: \n", len(valid_news), "news:", (t2-t1)/60, "min")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

Time to clean, tokenize and stem train data: 
 10240 news: 0.13447338740030926 min


Time to clean, tokenize and stem test data: 
 1267 news: 0.01324311097462972 min


Time to clean, tokenize and stem valid data: 
 1284 news: 0.01279436747233073 min


### [Google News corpus word2vec](http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/)

### Spell Check 

-  You can download the pre-trained model [**here**](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)

- Or clone it from GitHub [**GoogleNews-vectors-negative300**](https://github.com/mmihaltz/word2vec-GoogleNews-vectors)

> It’s 1.5GB! It includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset. The vector length is 300 features.

**3 million words * 300 features * 4bytes/feature = ~3.35GB**

> This file consist of the word2vec -  pre-trained Google News corpus (3 billion running words) to word vector model (3 million 300-dimension English word vectors).

> Look at the [**vocabulory list**](https://github.com/chrisjmccormick/inspect_word2vec/tree/master/vocabulary) used to train this model. Each text file contains 100,000 entries from the model. 


>  There are few things that this dataset contains and not. It has stop words like  “the”, “also”, “should” and does not have stop words like “a”, “and”, “of”. As I have removed the stop words the complexity is reduced as there is no need to check the spelling for stop words. 

> It does have numbers but in the form of entried wiht #. e.g., you won’t find “100”. But it does include entries like “###MHz_DDR2_SDRAM”. 

The model used [**WinPython-64bit-2.7.10.3**](https://winpython.github.io/) for efficient python distribution on Windows system. Helps to run the scripts in batches. 

In [ ]:
# model = gensim.models.KeyedVectors.load_word2vec_format('input_data/GoogleNews-vectors-negative300.bin.gz', binary=True)
model = gensim.models.KeyedVectors.load_word2vec_format((file_path_data + '/GoogleNews-vectors-negative300.bin.gz'), binary=True)
words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [ ]:
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
def spell_checker(text):
    all_words = re.findall(r'\w+', text.lower()) # split sentence to words
    spell_checked_text  = []
    for i in range(len(all_words)):
        spell_checked_text.append(correction(all_words[i]))
    return ' '.join(spell_checked_text)

In [ ]:
print("Before: \n", train_news['clean'][0] )
t1 = time.time()
train_news['clean'] = train_news['clean'].apply(spell_checker)
t2 = time.time()
print("\nTime to spell check the train data: \n", len(train_news), "news:", (t2-t1)/60, "min")

print("\nAfter: \n",train_news['clean'][0] )
train_news.head(5)

In [ ]:
t1 = time.time()
test_news['clean'] = test_news['clean'].apply(spell_checker)
test_news.head(5)
t2 = time.time()
print("\nTime to spell check the test data: \n", len(test_news), "news:", (t2-t1)/60, "min")

In [ ]:
t1 = time.time()
valid_news['clean'] = valid_news['clean'].apply(spell_checker)
valid_news.head(5)
t2 = time.time()
print("\nTime to spell check the valid data: \n", len(valid_news), "news:", (t2-t1)/60, "min")

##### Saved the trained dataset into a seperate CSV file 

In [ ]:
# train_news.to_csv("input_data/train_processed.csv", sep=',')
# test_news.to_csv("input_data/test_processed.csv", sep=',')
# valid_news.to_csv("input_data/valid_processed.csv", sep=',')
train_news.to_csv((file_path_data + '/train_processed.csv'), sep=',')
test_news.to_csv((file_path_data + '/test_processed.csv'), sep=',')
valid_news.to_csv((file_path_data + '/valid_processed.csv'), sep=',')

### Visualization 

#### WordCloud 

> As a tool for visualization by using the frequency of words appeared in text, we use WordCloud. Note that it can give more information and insight of texts by analyzing correlations and similarities between words rather than analyzing texts only by the frequency of words appeared; however, it can give you some general shape of what this text is about quickly and intuitively.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline

In [ ]:
def cloud(data,backgroundcolor = 'white', width = 800, height = 600):
    wordcloud = WordCloud(stopwords = STOPWORDS, background_color = backgroundcolor,
                         width = width, height = height).generate(data)
    plt.figure(figsize = (15, 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()
    
cloud(' '.join(train_news['clean']))

In [ ]:
cloud(' '.join(test_news['clean']))

In [ ]:
cloud(' '.join(valid_news['clean']))

#### Inferences from visulaization: 
- The large words are the words that are frequently appeared in the text/corpus

# Feature: Corpus Structure

### Import necessary Libraries

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics

### Import Data Set

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

train_news = pd.read_csv((file_path_data + '/train.tsv'), sep='\t', names = colnames, error_bad_lines=False)
test_news = pd.read_csv((file_path_data + '/test.tsv'), sep='\t', names = colnames, error_bad_lines=False)
valid_news = pd.read_csv((file_path_data + '/valid.tsv'), sep='\t', names = colnames, error_bad_lines=False)

## New Data

In [ ]:
# Data Cleaning has been performed above
trump_tweets

In [ ]:
trump_tweets['date'] = pd.to_datetime(trump_tweets['date'])

In [ ]:
trump_tweets.sort_values(by='date')

In [ ]:
trump_tweets['Binary Label'] = 1

# News Scrapper

In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 20

upperframe=[]  
for page in range(1,pagesToGet+1):
    # print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    # print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    # print(len(links))
    filename="NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
politifact_df=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
politifact_df.head()

In [ ]:
pd.unique(politifact_df['Label'])

### Data Cleaning

In [ ]:
train_news['clean'] = train_news["headline_text"].apply(cleaning) 
                
test_news['clean'] = test_news["headline_text"].apply(cleaning)

valid_news['clean'] = valid_news["headline_text"].apply(cleaning)

trump_tweets['clean'] = trump_tweets["text"].apply(cleaning)

politifact_df['claen'] = politifact_df["Statement"].apply(cleaning)

In [ ]:
train_news.merge(trump_tweets)
train_news.merge(politifact_df)

In [ ]:
class CorpusStructure():
    
    def __init__(self):
        
        self.logistic_regression_pipeline = ''
        self.random_forest_pipeline = ''
        
    
    def trueFalseLabel(self, r):
        '''
        Consider Original --	True, Mostly-tru, Half-true as True (1), 
        and Barely-true, False, Pants-fire, full-flop, half-flip as False (0)
        '''

        v = r['label']

        if (v == 'true'):
            return 1
        if (v == 'mostly-true'):
            return 1
        if (v == 'half-true'):
            return 1
        if (v == 'barely-true'):
            return 0
        if (v == 'half-flip'):
            return 0
        if (v == 'full-flop'):
            return 0
        if (v == 'false'):
            return 0
        if (v == 'pants-fire'):
            return 0
    
    def logistic_regression_pipline(self, ngram_rg=(1,1)):
        '''
        Use TFiDF and Count Vector 
        '''
        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_rg)
        count_vectorizer = CountVectorizer(ngram_range=ngram_rg)
        
        X = count_vectorizer.fit_transform(train_news['clean'])
        # Testing the ngram generation:
        print("Get first 20 feature names:\n")
        print(count_vectorizer.get_feature_names()[: 20])
        
        X_train = tfidf_vectorizer.fit_transform(train_news['clean'].values)
        X_test = tfidf_vectorizer.transform(test_news['clean'].values)
        
        train_news['Binary Label'] = train_news.apply(lambda row: self.trueFalseLabel(row), axis=1)
        test_news['Binary Label'] = train_news.apply(lambda row: self.trueFalseLabel(row), axis=1)

        y_train = train_news['Binary Label']
        y_test = test_news['Binary Label']
        
        self.logistic_regression_pipeline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', LogisticRegression(penalty='l2', C=98.684210526315795))
            ])

        # print(X_train.shape)
        # print(y_train.shape)

        self.logistic_regression_pipeline.fit(train_news['clean'].values, y_train)
        predicts = self.logistic_regression_pipeline.predict(test_news['clean'].values)
        score = metrics.accuracy_score(y_test, predicts)
        print("Accuracy:   %0.5f" % score)
        
        
    def logistic_regression_predict(self, text):
        predicted = self.logistic_regression_pipeline.predict([text])
        predicedProb = self.logistic_regression_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    
    def random_forest_pipline(self, ngram_rg=(2, 2)):
                
        tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=ngram_rg)
        count_vectorizer = CountVectorizer(ngram_range=ngram_rg)
        
        X = count_vectorizer.fit_transform(train_news['clean'])
        
        X_train = tfidf_vectorizer.fit_transform(train_news['clean'].values)
        X_test = tfidf_vectorizer.transform(test_news['clean'].values)
        
        train_news['Binary Label'] = train_news.apply(lambda row: self.trueFalseLabel(row), axis=1)
        test_news['Binary Label'] = train_news.apply(lambda row: self.trueFalseLabel(row), axis=1)

        y_train = train_news['Binary Label']
        y_test = test_news['Binary Label']
        
        self.random_forest_pipline = Pipeline([
            ('vectorizer', tfidf_vectorizer),
            ('clf', RandomForestClassifier(n_estimators = 300))
            ])


        self.random_forest_pipline.fit(train_news['clean'].values, y_train)
        predicts = self.random_forest_pipline.predict(test_news['clean'].values)
        score = metrics.accuracy_score(y_test, predicts)
        # Model Accuracy, how often is the classifier correct?
        print("Accuracy:",metrics.accuracy_score(y_test, predicts))

    def random_forest_predict(self, text):
        predicted = self.random_forest_pipeline.predict([text])
        predicedProb = self.random_forest_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    


In [ ]:
corpus_structure = CorpusStructure()

In [ ]:
for i in range(5):
  corpus_structure.logistic_regression_pipline(ngram_rg=(i, i))
  print("\n================\n")

In [ ]:
corpus_structure.random_forest_pipline(ngram_rg=(2, 2))

# Reference

1. https://alternusvera.wordpress.com/
2. https://devopedia.org/text-corpus-for-nlp#summary
3. https://www.corpusdata.org/
4. [Twitter Sentiment Analysis of Trump’s Coronavirus Response](https://medium.com/swlh/coronavirus-python-tutorial-1-520cc960aac1)

1.  https://medium.com/glose-team/how-to-evaluate-text-readability-with-nlp-9c04bd3f46a2
2. https://scholar.smu.edu/cgi/viewcontent.cgi?article=1202&context=datasciencereview
3. https://en.wikipedia.org/wiki/Readability
4. Google Search: nlp reading level detection
5. https://blog.insightdatascience.com/verbiage-using-nlp-to-improve-k-12-content-marketing-8906d2810fda
6. https://www.geeksforgeeks.org/readability-index-pythonnlp/
7. [Scraping Data Off Twitter Using Python | Twitterscraper + NLP + Data Visualization](https://www.youtube.com/watch?v=MpIi4HtCiVk&t=81s)
8. [The Generative Style Transformer](https://medium.com/agara-labs/the-generative-style-transformer-3564bce04d04)
9. [Two minutes NLP — Quick intro to Text Style Transfer](https://medium.com/nlplanet/two-minutes-nlp-quick-intro-to-text-style-transfer-61de9cbd4083)
10. [Reinforcement Learning Based Text Style Transfer without Parallel Training Corpus](https://www.youtube.com/watch?v=r8Cn_0Nb-ng)
11. [Scraping 1000’s of News Articles using 10 simple steps
](https://towardsdatascience.com/scraping-1000s-of-news-articles-using-10-simple-steps-d57636a49755)
12.

# Dataset

1. [Trump Twitter Archive](https://www.thetrumparchive.com/)